# Modelo de Gordon - FIIs
### Recomendação: FIIs de Tijolos

In [1]:
# Bibliotecas
import numpy as np
import pandas as pd
import requests
import yfinance as yf

In [2]:
# Fundos Imobiliários 
list_fiis=['XPLG11','HGLG11','HGRE11','SDIL11']

In [3]:
# Obtendo Dividendos (Fonte: fundamentus.com.br)
list_df = []
dic_sum12 = {}
for fii in list_fiis:
    url = f'http://www.fundamentus.com.br/fii_proventos.php?papel={fii}&tipo=2'
    header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"}    
    r = requests.get(url, headers=header)
    df = pd.read_html(r.text,  decimal=',', thousands='.')[0]
    df['Data de Pagamento'] = df['Data de Pagamento'].apply(pd.to_datetime,format='%d/%m/%Y')
    df = df[['Data de Pagamento','Valor']].set_index('Data de Pagamento')    
    df.index.rename('Data',inplace=True)
    df.rename(columns={'Valor':fii},inplace=True)
    list_df.append(df)
    dic_sum12[fii] = round(df[fii].head(12).sum(),4)

df_fii = pd.concat(list_df,axis=1)
df_fii.index = df_fii.index.sort_values()

In [4]:
# Obtendo Valores dos Títulos do Tesouro Direto
url_td = 'https://www.tesourotransparente.gov.br/ckan/dataset/df56aa42-484a-4a59-8184-7676580c81e3/resource/796d2059-14e9-44e3-80c9-2d9e30b405c1/download/PrecoTaxaTesouroDireto.csv'
df_td_raw = pd.read_csv(url_td,sep=';')

# Limpando os Dados e Gerando um DF com as Taxas de Compra (IPCA+)
df_td = df_td_raw[df_td_raw['Tipo Titulo'].isin(['Tesouro IPCA+','Tesouro IPCA+ com Juros Semestrais'])].drop(columns=['Taxa Venda Manha','PU Compra Manha','PU Venda Manha','PU Base Manha'])
df_td[['Data Base','Data Vencimento']] = df_td[['Data Base','Data Vencimento']].apply(pd.to_datetime,format='%d/%m/%Y')
df_td = df_td.sort_values(by=['Data Base','Data Vencimento'])
df_td = df_td.pivot_table(values='Taxa Compra Manha', index=['Data Base','Data Vencimento'],columns='Tipo Titulo',aggfunc='first')

In [43]:
# Obtendo Dados sobre a Expectativa do Mercado para a Inflação nos próximos 12 Meses
url_ex = "https://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/odata/ExpectativasMercadoInflacao12Meses?$orderby=Data&$format=text/csv&$select=Indicador,Data,Media,Mediana,Minimo,Maximo,numeroRespondentes"
df_ex_raw = pd.read_csv(url_ex,sep=',')
df_ex = df_ex_raw[df_ex_raw['Indicador']=='IPCA']
df_ex.rename(columns={'Indicador': 'Expectativa do Mercado'},inplace=True)


In [44]:
display(df_fii.tail(10)) # DF com os últimos dividendos 
display(df_td.tail(8)) # DF com as taxas do tesouro direto
display(df_ex.tail(5)) # DF com com a expectativa do mercado para a inflação 
display(dic_sum12) # Dicionário com a soma dos últimos 12 dividendos

,XPLG11,HGLG11,HGRE11,SDIL11
Data,,,,
2020-07-14,NaN,0.78,0.57,NaN
2020-07-15,0.57,NaN,NaN,0.44
2020-08-14,0.58,0.78,0.65,0.50
2020-09-15,0.58,0.78,0.65,0.55
2020-10-15,0.58,0.60,0.65,0.55
2020-11-13,0.58,NaN,NaN,0.59
2020-11-16,NaN,0.78,0.65,NaN
2020-12-14,0.58,0.78,0.65,NaN
2020-12-15,NaN,NaN,NaN,0.60


Tipo Titulo                Tesouro IPCA+ Tesouro IPCA+ com Juros Semestrais
Data Base  Data Vencimento                                                 
2021-01-18 2024-08-15               1,97                               1,88
           2026-08-15               2,74                               2,60
           2030-08-15                NaN                               3,17
           2035-05-15               3,60                               3,42
           2040-08-15                NaN                               3,72
           2045-05-15               3,60                               3,96
           2050-08-15                NaN                               3,97
           2055-05-15                NaN                               3,99

,Expectativa do Mercado,Data,Media,Mediana,Minimo,Maximo,numeroRespondentes
80425,IPCA,2021-01-14,"3,51","3,48","2,5","4,2",54.0
80433,IPCA,2021-01-15,"3,49","3,43","2,68","4,26",120.0
80434,IPCA,2021-01-15,"3,52","3,47","2,89","4,26",99.0
80442,IPCA,2021-01-15,"3,49","3,43","2,63","4,25",97.0
80443,IPCA,2021-01-15,"3,52","3,47","2,89","4,25",81.0


{'XPLG11': 7.01, 'HGLG11': 11.03, 'HGRE11': 8.06, 'SDIL11': 6.23}

In [47]:
# Modelo de Gordon
d = {'XPLG11': 7.01, 'HGLG11': 10.50, 'HGRE11': 9.06, 'SDIL11': 6.23}       # Dividendos esperado para os próximos 12 meses
ipca = 3.52/100                                                             # IPCA esperado para os próximos 12 meses
valor_risco = 2.5/100                                                       # Valor de risco 
g = 0                                                                       # Crescimento esperado
k = ipca + valor_risco
vp = {}
for fii in d:
    vp[fii] = round(d[fii]/(k+g),2)

# Gerando DF com os Preços Atuais e Valores gerados pelo Modelo     
list_price=[]
for fii in list_fiis:
    df = pd.DataFrame()
    ticker = yf.Ticker(f"{fii}.SA")
    df[fii] = ticker.history(period='1day')['Close']
    list_price.append(df)

df_price = pd.concat(list_price,axis=1)
df_price.index = df_price.index.strftime(date_format="%d/%m/%Y")
df_price = df_price.append(pd.DataFrame(vp,index=['Gordon'],columns=df_price.columns))
df_price 

,XPLG11,HGLG11,HGRE11,SDIL11
19/01/2021,120.30,181.50,150.15,103.12
Gordon,116.45,174.42,150.50,103.49
